In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
from nltk import word_tokenize
from nltk.stem.porter import *
from nltk.tokenize import RegexpTokenizer
import os,sys
import re
import pickle
import pandas as pd
import numpy as np
import tqdm
from tqdm import tqdm
import random
import math
from collections import Counter
import functools
import time , heapq
# from itertools import * #Functions creating iterators for efficient looping

In [2]:
with open('data/stop_words.txt') as f:
    stop_words_list = f.read().splitlines() #stop_list1
stop_list2 = pickle.load(open('data/stop_list2.pkl','rb'))
ps = PorterStemmer() # Stemming
stop_words = set(stopwords.words('english')) #Stopword
short = ['.', ',', '"', "\'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}', "'at",
         "_","`","\'\'","--","``",".,","//",":","___",'_the','-',"'em",".com",
                   '\'s','\'m','\'re','\'ll','\'d','n\'t','shan\'t',"...","\'ve",'u']
stop_words_list.extend(short)
stop_words_list.extend(stop_list2)
stop_words.update(stop_words_list)

In [3]:
def preprocess(texts):
    tokens = [i for i in word_tokenize(texts.lower()) if i not in stop_words]  # Tokenization.# Lowercasing
    token_result = ''
    token_result_ = ''
    for i,token in enumerate(tokens): #list2str
        token_result += ps.stem(token) + ' '
    token_result = ''.join([i for i in token_result if not i.isdigit()])
    token_result = [i for i in word_tokenize(token_result) if i not in stop_words]
    for i,token in enumerate(token_result):
        token_result_ += token + ' '
    return token_result_

In [4]:
def cosine(DOCx, DOCy):
    '''
    input: doc1 path name(str) , doc2 path name(str)
    ouput: two doc's cosine similarity
    '''
    dfx = pd.read_csv(DOCx,sep=' ',names=['tindex','tfidf'],header=None)
    dfy = pd.read_csv(DOCy,sep=' ',names=['tindex','tfidf'],header=None)
    dfx = dfx.drop(0)
    dfy = dfy.drop(0)
    dfxy = pd.merge(dfx,dfy,on='tindex',how='outer')
    dfxy.fillna(0,inplace=True)
    up = sum(dfxy.tfidf_x * dfxy.tfidf_y)
    down = np.sqrt(sum(np.square(dfxy.tfidf_x)))* np.sqrt(sum(np.square(dfxy.tfidf_y)))
    result = up / down
    return result

In [111]:
def merge_sim(C, j, i, m):
    a = np.max([C[j][i], C[j][m]])
    b = np.min([C[j][i], C[j][m]])
    return (a+b)/2 # max: single，min:complete

In [112]:
def heap_merge_sim(C, j, i, m):
    a = np.min([C[j][i][0], C[j][m][0]])
    b = np.max([C[j][i][0], C[j][m][0]])
    return (a+b)/2#max 取不相似的 #min:取相似的(single)

In [14]:
def cosine_sim(x, y):
    return np.dot(x, y)

In [ ]:
N=1095
C=np.zeros([N,N])
I=np.ones((N,), dtype=int)
eps=1e-10
for i in tqdm(range(N)):
#     c=[]
#     I[i]=1
    for j in range(N-i-1):
#         if i == j:
#             cos=-1
#         else:
        cos = cosine('data/tf-idf/'+str(i+1)+'.txt','data/tf-idf/'+str(j+i+2)+'.txt')
        C[i][j+i+1]=C[j+i+1][i]=cos+eps
#         c.append(cos)
#     C.append(c)
# C = np.array(C)
print(C.shape)
pickle.dump(obj=C,file=open('data/C.pkl','wb'))

  1%|          | 11/1095 [00:56<1:32:30,  5.12s/it]

In [113]:
N=1095
I=np.ones((N,), dtype=int)
eps=1e-10

cluster_representations = []
simple_cluster_results = []
C = pickle.load(open('data/C.pkl','rb'))
Ks = [8, 13, 20]
for n in range(N):
    cluster_representations.append([n])

In [114]:
simple_HAC_stime = time.time()
simple_A = []
for k in tqdm(range(N - 1)):
    
    # print('Simple HAC', k)
    
    # print(C)
    # print('\n')
    
    max_sim = 0
    max_i = 0
    max_m = 0
    for i in range(N): #去找最大相似度的df<i,m><-argmax
        for m in range(i + 1):
            if i != m and I[i] == 1 and I[m] == 1 and C[i][m] >= max_sim:
                max_sim = C[i][m]
                max_i = i
                max_m = m
                
    simple_A.append((max_i, max_m))
    
#     print(max_sim)
    
    cluster_representations[max_i] += cluster_representations[max_m] #把m並進去i
    cluster_representations[max_m] = None
    
    for j in range(N): #更新全部文章對i的similarity
        the_sim = merge_sim(C, j, max_i, max_m)
        C[max_i][j] = the_sim
        C[j][max_i] = the_sim
        
    I[max_m] = 0 #m已經合併
    
    if np.sum(I) in Ks: #如果現在還活著的個數有再想要的分群當中

        the_cluster_result = sorted([sorted(cluster) for cluster in cluster_representations if cluster is not None])
        
        # print(A)
        # print(I)
        # print(cluster_representations)
        # print(the_cluster_result)
        simple_cluster_results.append(the_cluster_result)


print('Simple HAC Time Taken', time.time() - simple_HAC_stime)
simple_cluster_results



  0%|          | 0/1094 [00:00<?, ?it/s]

  0%|          | 1/1094 [00:00<09:06,  2.00it/s]

  0%|          | 2/1094 [00:00<08:54,  2.04it/s]

  0%|          | 3/1094 [00:01<08:49,  2.06it/s]

  0%|          | 4/1094 [00:01<08:47,  2.07it/s]

  0%|          | 5/1094 [00:02<08:46,  2.07it/s]

  1%|          | 6/1094 [00:02<08:46,  2.07it/s]

  1%|          | 7/1094 [00:03<08:45,  2.07it/s]

  1%|          | 8/1094 [00:03<08:45,  2.07it/s]

  1%|          | 9/1094 [00:04<08:45,  2.06it/s]

  1%|          | 10/1094 [00:04<08:44,  2.07it/s]

  1%|          | 11/1094 [00:05<08:43,  2.07it/s]

  1%|          | 12/1094 [00:05<08:43,  2.07it/s]

  1%|          | 13/1094 [00:06<08:42,  2.07it/s]

  1%|▏         | 14/1094 [00:06<08:42,  2.07it/s]

  1%|▏         | 15/1094 [00:07<08:41,  2.07it/s]

  1%|▏         | 16/1094 [00:07<08:41,  2.07it/s]

  2%|▏         | 17/1094 [00:08<08:40,  2.07it/s]

  2%|▏         | 18/1094 [00:08<08:39,  2.07it/s]

  2%|▏         | 19/1094 [00:09<08:37,  2.08it/

 13%|█▎        | 143/1094 [01:04<07:11,  2.21it/s]

 13%|█▎        | 144/1094 [01:05<07:10,  2.21it/s]

 13%|█▎        | 145/1094 [01:05<07:09,  2.21it/s]

 13%|█▎        | 146/1094 [01:06<07:09,  2.21it/s]

 13%|█▎        | 147/1094 [01:06<07:08,  2.21it/s]

 14%|█▎        | 148/1094 [01:06<07:07,  2.21it/s]

 14%|█▎        | 149/1094 [01:07<07:07,  2.21it/s]

 14%|█▎        | 150/1094 [01:07<07:06,  2.21it/s]

 14%|█▍        | 151/1094 [01:08<07:05,  2.21it/s]

 14%|█▍        | 152/1094 [01:08<07:05,  2.22it/s]

 14%|█▍        | 153/1094 [01:09<07:04,  2.22it/s]

 14%|█▍        | 154/1094 [01:09<07:03,  2.22it/s]

 14%|█▍        | 155/1094 [01:09<07:03,  2.22it/s]

 14%|█▍        | 156/1094 [01:10<07:02,  2.22it/s]

 14%|█▍        | 157/1094 [01:10<07:01,  2.22it/s]

 14%|█▍        | 158/1094 [01:11<07:01,  2.22it/s]

 15%|█▍        | 159/1094 [01:11<07:00,  2.22it/s]

 15%|█▍        | 160/1094 [01:11<06:59,  2.22it/s]

 15%|█▍        | 161/1094 [01:12<06:59,  2.23it/s]

 15%|█▍     

 26%|██▌       | 284/1094 [02:00<05:44,  2.35it/s]

 26%|██▌       | 285/1094 [02:01<05:43,  2.35it/s]

 26%|██▌       | 286/1094 [02:01<05:43,  2.35it/s]

 26%|██▌       | 287/1094 [02:01<05:42,  2.36it/s]

 26%|██▋       | 288/1094 [02:02<05:41,  2.36it/s]

 26%|██▋       | 289/1094 [02:02<05:41,  2.36it/s]

 27%|██▋       | 290/1094 [02:02<05:40,  2.36it/s]

 27%|██▋       | 291/1094 [02:03<05:40,  2.36it/s]

 27%|██▋       | 292/1094 [02:03<05:39,  2.36it/s]

 27%|██▋       | 293/1094 [02:04<05:39,  2.36it/s]

 27%|██▋       | 294/1094 [02:04<05:38,  2.36it/s]

 27%|██▋       | 295/1094 [02:04<05:37,  2.36it/s]

 27%|██▋       | 296/1094 [02:05<05:37,  2.37it/s]

 27%|██▋       | 297/1094 [02:05<05:36,  2.37it/s]

 27%|██▋       | 298/1094 [02:05<05:36,  2.37it/s]

 27%|██▋       | 299/1094 [02:06<05:35,  2.37it/s]

 27%|██▋       | 300/1094 [02:06<05:34,  2.37it/s]

 28%|██▊       | 301/1094 [02:06<05:34,  2.37it/s]

 28%|██▊       | 302/1094 [02:07<05:33,  2.37it/s]

 28%|██▊    

 39%|███▉      | 425/1094 [02:49<04:26,  2.51it/s]

 39%|███▉      | 426/1094 [02:49<04:26,  2.51it/s]

 39%|███▉      | 427/1094 [02:50<04:25,  2.51it/s]

 39%|███▉      | 428/1094 [02:50<04:25,  2.51it/s]

 39%|███▉      | 429/1094 [02:50<04:24,  2.51it/s]

 39%|███▉      | 430/1094 [02:50<04:24,  2.51it/s]

 39%|███▉      | 431/1094 [02:51<04:23,  2.52it/s]

 39%|███▉      | 432/1094 [02:51<04:23,  2.52it/s]

 40%|███▉      | 433/1094 [02:51<04:22,  2.52it/s]

 40%|███▉      | 434/1094 [02:52<04:22,  2.52it/s]

 40%|███▉      | 435/1094 [02:52<04:21,  2.52it/s]

 40%|███▉      | 436/1094 [02:52<04:20,  2.52it/s]

 40%|███▉      | 437/1094 [02:53<04:20,  2.52it/s]

 40%|████      | 438/1094 [02:53<04:19,  2.52it/s]

 40%|████      | 439/1094 [02:53<04:19,  2.52it/s]

 40%|████      | 440/1094 [02:54<04:18,  2.53it/s]

 40%|████      | 441/1094 [02:54<04:18,  2.53it/s]

 40%|████      | 442/1094 [02:54<04:17,  2.53it/s]

 40%|████      | 443/1094 [02:55<04:17,  2.53it/s]

 41%|████   

 52%|█████▏    | 566/1094 [03:32<03:17,  2.67it/s]

 52%|█████▏    | 567/1094 [03:32<03:17,  2.67it/s]

 52%|█████▏    | 568/1094 [03:32<03:16,  2.67it/s]

 52%|█████▏    | 569/1094 [03:33<03:16,  2.67it/s]

 52%|█████▏    | 570/1094 [03:33<03:16,  2.67it/s]

 52%|█████▏    | 571/1094 [03:33<03:15,  2.67it/s]

 52%|█████▏    | 572/1094 [03:33<03:15,  2.67it/s]

 52%|█████▏    | 573/1094 [03:34<03:14,  2.68it/s]

 52%|█████▏    | 574/1094 [03:34<03:14,  2.68it/s]

 53%|█████▎    | 575/1094 [03:34<03:13,  2.68it/s]

 53%|█████▎    | 576/1094 [03:34<03:13,  2.68it/s]

 53%|█████▎    | 577/1094 [03:35<03:12,  2.68it/s]

 53%|█████▎    | 578/1094 [03:35<03:12,  2.68it/s]

 53%|█████▎    | 579/1094 [03:35<03:11,  2.68it/s]

 53%|█████▎    | 580/1094 [03:36<03:11,  2.68it/s]

 53%|█████▎    | 581/1094 [03:36<03:11,  2.69it/s]

 53%|█████▎    | 582/1094 [03:36<03:10,  2.69it/s]

 53%|█████▎    | 583/1094 [03:36<03:10,  2.69it/s]

 53%|█████▎    | 584/1094 [03:37<03:09,  2.69it/s]

 53%|█████▎ 

 65%|██████▍   | 707/1094 [04:09<02:16,  2.83it/s]

 65%|██████▍   | 708/1094 [04:09<02:16,  2.84it/s]

 65%|██████▍   | 709/1094 [04:09<02:15,  2.84it/s]

 65%|██████▍   | 710/1094 [04:10<02:15,  2.84it/s]

 65%|██████▍   | 711/1094 [04:10<02:14,  2.84it/s]

 65%|██████▌   | 712/1094 [04:10<02:14,  2.84it/s]

 65%|██████▌   | 713/1094 [04:10<02:14,  2.84it/s]

 65%|██████▌   | 714/1094 [04:11<02:13,  2.84it/s]

 65%|██████▌   | 715/1094 [04:11<02:13,  2.84it/s]

 65%|██████▌   | 716/1094 [04:11<02:12,  2.85it/s]

 66%|██████▌   | 717/1094 [04:11<02:12,  2.85it/s]

 66%|██████▌   | 718/1094 [04:12<02:12,  2.85it/s]

 66%|██████▌   | 719/1094 [04:12<02:11,  2.85it/s]

 66%|██████▌   | 720/1094 [04:12<02:11,  2.85it/s]

 66%|██████▌   | 721/1094 [04:12<02:10,  2.85it/s]

 66%|██████▌   | 722/1094 [04:13<02:10,  2.85it/s]

 66%|██████▌   | 723/1094 [04:13<02:09,  2.85it/s]

 66%|██████▌   | 724/1094 [04:13<02:09,  2.86it/s]

 66%|██████▋   | 725/1094 [04:13<02:09,  2.86it/s]

 66%|██████▋

 78%|███████▊  | 848/1094 [04:41<01:21,  3.01it/s]

 78%|███████▊  | 849/1094 [04:42<01:21,  3.01it/s]

 78%|███████▊  | 850/1094 [04:42<01:21,  3.01it/s]

 78%|███████▊  | 851/1094 [04:42<01:20,  3.01it/s]

 78%|███████▊  | 852/1094 [04:42<01:20,  3.01it/s]

 78%|███████▊  | 853/1094 [04:42<01:19,  3.01it/s]

 78%|███████▊  | 854/1094 [04:43<01:19,  3.02it/s]

 78%|███████▊  | 855/1094 [04:43<01:19,  3.02it/s]

 78%|███████▊  | 856/1094 [04:43<01:18,  3.02it/s]

 78%|███████▊  | 857/1094 [04:43<01:18,  3.02it/s]

 78%|███████▊  | 858/1094 [04:44<01:18,  3.02it/s]

 79%|███████▊  | 859/1094 [04:44<01:17,  3.02it/s]

 79%|███████▊  | 860/1094 [04:44<01:17,  3.02it/s]

 79%|███████▊  | 861/1094 [04:44<01:17,  3.02it/s]

 79%|███████▉  | 862/1094 [04:44<01:16,  3.03it/s]

 79%|███████▉  | 863/1094 [04:45<01:16,  3.03it/s]

 79%|███████▉  | 864/1094 [04:45<01:15,  3.03it/s]

 79%|███████▉  | 865/1094 [04:45<01:15,  3.03it/s]

 79%|███████▉  | 866/1094 [04:45<01:15,  3.03it/s]

 79%|███████

 90%|█████████ | 989/1094 [05:11<00:33,  3.18it/s]

 90%|█████████ | 990/1094 [05:11<00:32,  3.18it/s]

 91%|█████████ | 991/1094 [05:11<00:32,  3.18it/s]

 91%|█████████ | 992/1094 [05:11<00:32,  3.18it/s]

 91%|█████████ | 993/1094 [05:11<00:31,  3.18it/s]

 91%|█████████ | 994/1094 [05:12<00:31,  3.19it/s]

 91%|█████████ | 995/1094 [05:12<00:31,  3.19it/s]

 91%|█████████ | 996/1094 [05:12<00:30,  3.19it/s]

 91%|█████████ | 997/1094 [05:12<00:30,  3.19it/s]

 91%|█████████ | 998/1094 [05:12<00:30,  3.19it/s]

 91%|█████████▏| 999/1094 [05:12<00:29,  3.19it/s]

 91%|█████████▏| 1000/1094 [05:13<00:29,  3.19it/s]

 91%|█████████▏| 1001/1094 [05:13<00:29,  3.19it/s]

 92%|█████████▏| 1002/1094 [05:13<00:28,  3.20it/s]

 92%|█████████▏| 1003/1094 [05:13<00:28,  3.20it/s]

 92%|█████████▏| 1004/1094 [05:13<00:28,  3.20it/s]

 92%|█████████▏| 1005/1094 [05:14<00:27,  3.20it/s]

 92%|█████████▏| 1006/1094 [05:14<00:27,  3.20it/s]

 92%|█████████▏| 1007/1094 [05:14<00:27,  3.20it/s]

 92%

Simple HAC Time Taken 330.15829586982727


[[[0,
   1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   29,
   31,
   32,
   33,
   34,
   35,
   36,
   37,
   38,
   39,
   40,
   42,
   44,
   45,
   46,
   47,
   48,
   49,
   50,
   51,
   52,
   53,
   54,
   55,
   56,
   57,
   58,
   59,
   60,
   61,
   62,
   63,
   64,
   65,
   66,
   68,
   70,
   71,
   72,
   73,
   74,
   75,
   76,
   77,
   78,
   79,
   80,
   81,
   83,
   84,
   86,
   87,
   88,
   89,
   92,
   93,
   94,
   95,
   96,
   97,
   98,
   100,
   102,
   104,
   105,
   106,
   107,
   108,
   109,
   110,
   111,
   113,
   115,
   116,
   117,
   118,
   119,
   120,
   121,
   123,
   124,
   125,
   126,
   127,
   128,
   182,
   225,
   231,
   258,
   630,
   631,
   648,
   663,
   687,
   688,
   711,
   728],
  [10,
   18,
   28,
   112,
   114,
   168,
   261,
   277,
   295,
   300,
   303,
   307,
   315,
   316,
   

HEAP HAC
* 只需要simple的三分之一時間

In [115]:
heap_cluster_results = []

C_ = pickle.load(open('data/C.pkl','rb'))
C = []
P = []
I = np.zeros(N)

cluster_representations = []


N=1095
heap_HAC_stime = time.time()



for n in range(N):
    c = []
    for i in range(N):
#         the_neg_sim = -cosine_sim(tfidf_vecs[n], tfidf_vecs[i])
#         the_neg_sim -= eps * n * i
        the_sim = -C_[n][i] #- eps*n*i*2
        
        c.append([the_sim, i])
        
        #the_sim = cosine_sim(tfidf_vecs[n], tfidf_vecs[i])
        #c.append([the_sim, i])
        
    C.append(c)
        
    I[n] = 1
    C_list = sorted(C[n])
    C_list.remove(C[n][n])
    P.append(C_list)
    heapq.heapify(P[n])
    cluster_representations.append([n])
heap_A = [] # list of merges
    
for k in tqdm(range(N - 1)):

    
    min_neg_sim = 0
    min_i = 0
    min_m = 0
    
    for i in range(N):
        
        if I[i] == 1 and P[i][0][0] <= min_neg_sim: #similarity 最大的
            min_i = i
            min_neg_sim = P[i][0][0]
    
    
#     print(np.abs(min_neg_sim))
    
    min_m = P[min_i][0][1]
    
    k2, k1 = sorted([min_i, min_m])
    
            
    heap_A.append((k1, k2))
    
    # if simple_A[k] != heap_A[k]:
    #    print('not aligned with simple', k)
    
    I[k2] = 0 #k2並去k1
    
    P[k1] = []
    
    for i in range(N):
        if I[i] == 1 and i != k1: #for each i with OOXX
            
            # delete method could be improved
            P[i].remove(C[i][k1])
            P[i].remove(C[i][k2])
            
            heapq.heapify(P[i])
            
            the_sim = heap_merge_sim(C, i, k1, k2) #sim
            C[i][k1][0] = the_sim
            heapq.heappush(P[i], C[i][k1])
            C[k1][i][0] = the_sim
            heapq.heappush(P[k1], C[k1][i])
            
    
    cluster_representations[k1] += cluster_representations[k2]
    cluster_representations[k2] = None
        
    if np.sum(I) in Ks:

        the_cluster_result = sorted([sorted(cluster) for cluster in cluster_representations if cluster is not None])
        
        # print(A)
        # print(I)
        # print(cluster_representations)
        # print('the cluster result', the_cluster_result)
        heap_cluster_results.append(the_cluster_result)


print('Heap HAC_ Time Taken', time.time() - heap_HAC_stime)
heap_cluster_results



  0%|          | 0/1094 [00:00<?, ?it/s]

  0%|          | 1/1094 [00:00<03:34,  5.08it/s]

  0%|          | 2/1094 [00:00<03:38,  4.99it/s]

  0%|          | 3/1094 [00:00<03:32,  5.12it/s]

  0%|          | 4/1094 [00:00<03:32,  5.12it/s]

  0%|          | 5/1094 [00:00<03:35,  5.06it/s]

  1%|          | 6/1094 [00:01<03:34,  5.08it/s]

  1%|          | 7/1094 [00:01<03:34,  5.08it/s]

  1%|          | 8/1094 [00:01<03:34,  5.07it/s]

  1%|          | 9/1094 [00:01<03:36,  5.02it/s]

  1%|          | 10/1094 [00:02<03:38,  4.97it/s]

  1%|          | 11/1094 [00:02<03:39,  4.93it/s]

  1%|          | 12/1094 [00:02<03:38,  4.95it/s]

  1%|          | 13/1094 [00:02<03:38,  4.95it/s]

  1%|▏         | 14/1094 [00:02<03:38,  4.95it/s]

  1%|▏         | 15/1094 [00:03<03:35,  5.00it/s]

  1%|▏         | 16/1094 [00:03<03:35,  5.00it/s]

  2%|▏         | 17/1094 [00:03<03:34,  5.01it/s]

  2%|▏         | 18/1094 [00:03<03:34,  5.02it/s]

  2%|▏         | 19/1094 [00:03<03:33,  5.02it/

 13%|█▎        | 143/1094 [00:26<02:57,  5.36it/s]

 13%|█▎        | 144/1094 [00:26<02:57,  5.37it/s]

 13%|█▎        | 145/1094 [00:27<02:56,  5.37it/s]

 13%|█▎        | 146/1094 [00:27<02:56,  5.37it/s]

 13%|█▎        | 147/1094 [00:27<02:56,  5.38it/s]

 14%|█▎        | 148/1094 [00:27<02:55,  5.38it/s]

 14%|█▎        | 149/1094 [00:27<02:55,  5.38it/s]

 14%|█▎        | 150/1094 [00:27<02:55,  5.39it/s]

 14%|█▍        | 151/1094 [00:28<02:54,  5.39it/s]

 14%|█▍        | 152/1094 [00:28<02:54,  5.39it/s]

 14%|█▍        | 153/1094 [00:28<02:54,  5.40it/s]

 14%|█▍        | 154/1094 [00:28<02:54,  5.40it/s]

 14%|█▍        | 155/1094 [00:28<02:53,  5.40it/s]

 14%|█▍        | 156/1094 [00:28<02:53,  5.40it/s]

 14%|█▍        | 157/1094 [00:29<02:53,  5.41it/s]

 14%|█▍        | 158/1094 [00:29<02:53,  5.41it/s]

 15%|█▍        | 159/1094 [00:29<02:52,  5.41it/s]

 15%|█▍        | 160/1094 [00:29<02:52,  5.42it/s]

 15%|█▍        | 161/1094 [00:29<02:52,  5.42it/s]

 15%|█▍     

 26%|██▌       | 284/1094 [00:49<02:20,  5.78it/s]

 26%|██▌       | 285/1094 [00:49<02:19,  5.78it/s]

 26%|██▌       | 286/1094 [00:49<02:19,  5.79it/s]

 26%|██▌       | 287/1094 [00:49<02:19,  5.79it/s]

 26%|██▋       | 288/1094 [00:49<02:19,  5.79it/s]

 26%|██▋       | 289/1094 [00:49<02:18,  5.80it/s]

 27%|██▋       | 290/1094 [00:49<02:18,  5.80it/s]

 27%|██▋       | 291/1094 [00:50<02:18,  5.81it/s]

 27%|██▋       | 292/1094 [00:50<02:18,  5.81it/s]

 27%|██▋       | 293/1094 [00:50<02:17,  5.81it/s]

 27%|██▋       | 294/1094 [00:50<02:17,  5.82it/s]

 27%|██▋       | 295/1094 [00:50<02:17,  5.82it/s]

 27%|██▋       | 296/1094 [00:50<02:16,  5.83it/s]

 27%|██▋       | 297/1094 [00:50<02:16,  5.83it/s]

 27%|██▋       | 298/1094 [00:51<02:16,  5.83it/s]

 27%|██▋       | 299/1094 [00:51<02:16,  5.84it/s]

 27%|██▋       | 300/1094 [00:51<02:15,  5.84it/s]

 28%|██▊       | 301/1094 [00:51<02:15,  5.84it/s]

 28%|██▊       | 302/1094 [00:51<02:15,  5.85it/s]

 28%|██▊    

 39%|███▉      | 425/1094 [01:06<01:45,  6.37it/s]

 39%|███▉      | 426/1094 [01:06<01:44,  6.37it/s]

 39%|███▉      | 427/1094 [01:06<01:44,  6.38it/s]

 39%|███▉      | 428/1094 [01:07<01:44,  6.38it/s]

 39%|███▉      | 429/1094 [01:07<01:44,  6.39it/s]

 39%|███▉      | 430/1094 [01:07<01:43,  6.39it/s]

 39%|███▉      | 431/1094 [01:07<01:43,  6.39it/s]

 40%|███▉      | 433/1094 [01:07<01:43,  6.40it/s]

 40%|███▉      | 434/1094 [01:07<01:42,  6.41it/s]

 40%|███▉      | 435/1094 [01:07<01:42,  6.42it/s]

 40%|███▉      | 436/1094 [01:07<01:42,  6.42it/s]

 40%|███▉      | 437/1094 [01:08<01:42,  6.42it/s]

 40%|████      | 438/1094 [01:08<01:42,  6.43it/s]

 40%|████      | 439/1094 [01:08<01:41,  6.44it/s]

 40%|████      | 440/1094 [01:08<01:41,  6.44it/s]

 40%|████      | 441/1094 [01:08<01:41,  6.44it/s]

 40%|████      | 442/1094 [01:08<01:41,  6.45it/s]

 41%|████      | 444/1094 [01:08<01:40,  6.46it/s]

 41%|████      | 446/1094 [01:08<01:40,  6.47it/s]

 41%|████   

 65%|██████▍   | 710/1094 [01:26<00:46,  8.19it/s]

 65%|██████▌   | 713/1094 [01:26<00:46,  8.22it/s]

 65%|██████▌   | 716/1094 [01:26<00:45,  8.24it/s]

 66%|██████▌   | 719/1094 [01:27<00:45,  8.26it/s]

 66%|██████▌   | 722/1094 [01:27<00:44,  8.29it/s]

 66%|██████▋   | 726/1094 [01:27<00:44,  8.32it/s]

 67%|██████▋   | 729/1094 [01:27<00:43,  8.35it/s]

 67%|██████▋   | 733/1094 [01:27<00:43,  8.38it/s]

 67%|██████▋   | 737/1094 [01:27<00:42,  8.41it/s]

 68%|██████▊   | 741/1094 [01:27<00:41,  8.45it/s]

 68%|██████▊   | 745/1094 [01:27<00:41,  8.48it/s]

 68%|██████▊   | 749/1094 [01:27<00:40,  8.52it/s]

 69%|██████▉   | 753/1094 [01:28<00:39,  8.55it/s]

 69%|██████▉   | 757/1094 [01:28<00:39,  8.59it/s]

 70%|██████▉   | 761/1094 [01:28<00:38,  8.62it/s]

 70%|██████▉   | 765/1094 [01:28<00:38,  8.66it/s]

 70%|███████   | 770/1094 [01:28<00:37,  8.70it/s]

 71%|███████   | 775/1094 [01:28<00:36,  8.74it/s]

 71%|███████▏  | 780/1094 [01:28<00:35,  8.79it/s]

 72%|███████

Heap HAC_ Time Taken 93.42108201980591


[[[0,
   1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   29,
   31,
   32,
   33,
   34,
   35,
   36,
   37,
   38,
   39,
   40,
   42,
   44,
   45,
   46,
   47,
   48,
   49,
   50,
   51,
   52,
   53,
   54,
   55,
   56,
   57,
   58,
   59,
   60,
   61,
   62,
   63,
   64,
   65,
   66,
   68,
   70,
   71,
   72,
   73,
   74,
   75,
   76,
   77,
   78,
   79,
   80,
   81,
   83,
   84,
   86,
   87,
   88,
   89,
   92,
   93,
   94,
   95,
   96,
   97,
   98,
   100,
   102,
   104,
   105,
   106,
   107,
   108,
   109,
   110,
   111,
   113,
   115,
   116,
   117,
   118,
   119,
   120,
   121,
   123,
   124,
   125,
   126,
   127,
   128,
   182,
   225,
   231,
   258,
   630,
   631,
   648,
   663,
   687,
   688,
   711,
   728],
  [10,
   18,
   28,
   112,
   114,
   168,
   261,
   277,
   295,
   300,
   303,
   307,
   315,
   316,
   

In [104]:
C[i][k2]

[-0.2783121339081798, 0]

In [21]:
82.96472764015198 / 250.30226469039917

0.33145815817036933

In [116]:
for i in range(len(Ks)):
    print(i)
    if simple_cluster_results[i] != heap_cluster_results[i]:
        print('Warning')

0
1
2


In [84]:
with open('output/20_ans.txt','r') as f:
    w = f.readlines()
ans20=[]
temp=[]
for doc in w:
    doc_ = doc.split('\n')[0]
    if doc_ == '':
        ans20.append(temp)
        print(len(temp))
        temp=[]
    else:
        temp.append(int(doc_)-1)
print(len(temp))        
ans20.append(temp)

122
57
51
75
54
177
19
39
9
65
56
117
61
63
7
90
4
2
16
11


In [86]:
ans_all = []
ans_all.append(ans20)
ans_all.append(ans13)
ans_all.append(ans8)
len(ans8) , len(ans13) , len(ans20) , len(ans_all)

(8, 13, 20, 3)

In [96]:
simple_single = simple_cluster_results
heap_single = heap_cluster_results
len(simple_single[2]) , len(heap_single[2])

(8, 8)

In [108]:
simple_avg1 = simple_cluster_results #相乘的
heap_avg1 = heap_cluster_results
len(simple_avg1[2]) , len(heap_avg1[2])

(8, 8)

In [117]:
simple_avg2 = simple_cluster_results
heap_avg2 = heap_cluster_results
len(simple_avg2[2]) , len(heap_avg2[2])

(8, 8)

In [89]:
simple_complete = simple_cluster_results
heap_complete = heap_cluster_results
len(simple_complete[2]) , len(heap_complete[2])

(8, 8)

In [134]:
df = pd.read_csv('../HW3/voting_rev.csv')
clf13=[]
for i in range(13):
    
    temp = df[df.Value == (i+1)]['id'].tolist()
    temp.extend(df1[str(i+1)].tolist())
    temp = sorted(temp)
    clf13.append(temp)
len(clf13)

13

In [120]:
with open('../HW3/data/training.txt','r') as f:
    train_id = f.read().splitlines()
train_dict = {}
for trainid in train_id:
    trainid = trainid.split(' ')
    trainid = list(filter(None, trainid))
    train_dict[trainid[0]] = trainid[1:]
train_dict #class:doc_id
# train_dict = pickle.load(open('data/train_dict.pkl','rb'))

{'1': ['11',
  '19',
  '29',
  '113',
  '115',
  '169',
  '278',
  '301',
  '316',
  '317',
  '321',
  '324',
  '325',
  '338',
  '341'],
 '2': ['1',
  '2',
  '3',
  '4',
  '5',
  '6',
  '7',
  '8',
  '9',
  '10',
  '12',
  '13',
  '14',
  '15',
  '16'],
 '3': ['813',
  '817',
  '818',
  '819',
  '820',
  '821',
  '822',
  '824',
  '825',
  '826',
  '828',
  '829',
  '830',
  '832',
  '833'],
 '4': ['635',
  '680',
  '683',
  '702',
  '704',
  '705',
  '706',
  '708',
  '709',
  '719',
  '720',
  '722',
  '723',
  '724',
  '726'],
 '5': ['646',
  '751',
  '781',
  '794',
  '798',
  '799',
  '801',
  '812',
  '815',
  '823',
  '831',
  '839',
  '840',
  '841',
  '842'],
 '6': ['995',
  '998',
  '999',
  '1003',
  '1005',
  '1006',
  '1007',
  '1009',
  '1011',
  '1012',
  '1013',
  '1014',
  '1015',
  '1016',
  '1019'],
 '7': ['700',
  '730',
  '731',
  '732',
  '733',
  '735',
  '740',
  '744',
  '752',
  '754',
  '755',
  '756',
  '757',
  '759',
  '760'],
 '8': ['262',
  '296',
  '30

In [133]:
df1 = pd.DataFrame.from_dict(train_dict)
df1 = df1.astype('int')
df1['2'].tolist()

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16]

voting

In [684]:
# pickle.dump(obj=voting_lis , file=open('data/voting_lis.pkl','wb'))
# pickle.dump(obj=clf13 , file=open('data/clf13.pkl','wb'))
voting_lis = pickle.load(open('data/voting_lis.pkl','rb'))
clf13 = pickle.load(open('data/clf13.pkl'))

In [678]:
# voting_li8 = [ans_all , ans_all,ans_all,ans_all,ans_all,ans_all,ans_all,ans_all,ans_all,ans_all,
#              simple_complete,simple_complete,simple_complete,heap_complete,heap_complete,heap_complete,
#              simple_single ,simple_avg1,heap_avg1,simple_avg2 ,heap_avg2] #=>8
# voting_li20 = [ans_all , ans_all,simple_complete,simple_complete,simple_complete,simple_complete,
#              heap_complete,heap_complete,heap_complete,heap_complete,simple_single ,
#              simple_avg1   ,heap_avg1,simple_avg2,heap_avg2 ] #20
# voting_li13 = [ans_all ,simple_complete,heap_complete,simple_single ,
#              simple_avg1,heap_avg1,simple_avg2, heap_avg2] #13
# voting_lis = [voting_li20,voting_li13,voting_li8]

In [680]:
# voting_li = [ans_all , ans_all , simple_complete , heap_complete , simple_single , 
#              simple_avg1 ,heap_avg1, simple_avg2 ] #3154/1613/4332
# voting_li = [ans_all , ans_all , ans_all,simple_complete , heap_complete , simple_single , 
#              simple_avg1 ,heap_avg1, simple_avg2 ] #2073/2073/3929
# voting_li = [ans_all  ,simple_complete , heap_complete , simple_single , 
#              simple_avg1 ,heap_avg1, simple_avg2 ] #/2095/3871
# voting_li = [ans_all,ans_all  ,simple_complete , heap_complete , simple_single , heap_single,
#              simple_avg1 ,heap_avg1, simple_avg2,heap_avg2 ] #/1777/3730
# voting_li = [ans_all,ans_all  ,simple_complete , heap_complete ,
#              simple_avg1 ,heap_avg1, simple_avg2,heap_avg2 ] #/1359/4221
# voting_li = [ans_all , ans_all , ans_all,simple_complete , heap_complete , simple_single , 
#              simple_avg1 ,heap_avg1, simple_avg2,heap_avg2 ] #3215/1449/4243
# voting_li = [ans_all , ans_all ,simple_complete , heap_complete , simple_single , 
#              simple_avg1 ,heap_avg1, simple_avg2,heap_avg2 ] #1925/1925/4046
# voting_li = [ans_all , ans_all, ans_all, ans_all, ans_all, ans_all, ans_all, 
#              simple_complete ,simple_complete ,simple_complete ,simple_complete ,
#              heap_complete ,heap_complete ,heap_complete ,heap_complete ,
#              simple_single ,
#              simple_avg1   ,heap_avg1,
#              simple_avg2 ] #2230/3515=>578/1175
# voting_li = [ans_all , ans_all, ans_all, ans_all, ans_all, ans_all, 
#              simple_complete ,simple_complete ,simple_complete ,simple_complete ,
#              heap_complete ,heap_complete ,heap_complete ,heap_complete ,
#              simple_single ,
#              simple_avg1   ,heap_avg1,
#              simple_avg2 ] #2226/3417=>20:574/1105
# voting_li = [ans_all , ans_all,ans_all,ans_all,
#              simple_complete    ,simple_complete,simple_complete,simple_complete,
#              heap_complete    ,heap_complete,heap_complete,heap_complete,
#              simple_single ,
#              simple_avg1   ,heap_avg1,
#              simple_avg2 ] #2454/3384 =>20:683/1173
# voting_li = [ans_all ,
#              simple_complete    ,
#              heap_complete    ,
#              simple_single ,
#              simple_avg1   ,heap_avg1,
#              simple_avg2, heap_avg2] #2454/3384 =>20:683/1173

                #4: #5:/2062/3467，6://3592，7:/2125/3286

for i,voting_li in enumerate(voting_lis):
    pd8 = pd.DataFrame(0,index=range(1095),columns=range(8))
    pd13 = pd.DataFrame(0,index=range(1095),columns=range(13))
    pd20 = pd.DataFrame(0,index=range(1095),columns=range(20))
    for method in voting_li : #20,13,8
        sort20 = sorted(method[0],key=len)
        sort13 = sorted(method[1],key=len)
        sort8 = sorted(method[2],key=len)
        for c,cluster in enumerate(sort8): #8群
            for doc in cluster: #各群裡面成員
                pd8.loc[doc,c] +=1
        for c,cluster in enumerate(sort13):
            for doc in cluster:
                pd13.loc[doc,c] +=1
        for c,cluster in enumerate(sort20): #8群
            for doc in cluster: #各群裡面成員
                pd20.loc[doc,c] +=1
    sort13 = sorted(clf13,key=len)
    for c,cluster in enumerate(sort13):
        for doc in cluster:
            pd13.loc[doc,c] += int(len(voting_li)/1.3)
    if i ==0:
        df20 = pd20
        print(len(voting_li))
    elif i ==1:
        df13 = pd13
        print(len(voting_li))
    elif i == 2:
        df8 = pd8
        print(len(voting_li))

15
8
21


In [681]:
for df in [df8,df13,df20]:
    df['vote'] = 0
    for i in range(len(df)):
        df.loc[i,'vote'] = df.loc[i].idxmax()

In [682]:
counts = 0
counts_minimize = 0
for pd_ in [df8,df13,df20]:
    cols = pd_.columns.tolist()[:-1]
    count = 0
    count_minimize = 0
    for col in cols:
        count += len(pd_[pd_[col]>int(len(voting_li)/2 )])
    pd_2 = pd_.drop(['vote'],axis=1)
    for i in range(len(pd_2)):
        count_minimize += pd_2.loc[i].isin([pd_2.loc[i].max()]).sum()
    
    print(count,count_minimize)
    counts+=count
    counts_minimize += count_minimize
print("ALL:",counts,'Minimize:',counts_minimize)

697 1096
269 1095
381 1095
ALL: 1347 Minimize: 3286


In [683]:
for i in range(13):
    print(df13[df13['vote']==i].index.tolist())
print('======================================================================================')
for i in range(20):
    print(df20[df20['vote']==i].index.tolist())
print('======================================================================================')
for i in range(8):
    print(df8[df8['vote']==i].index.tolist())

[812, 816, 817, 818, 819, 820, 821, 823, 824, 825, 827, 828, 829, 831, 832, 833, 834, 836, 837, 842, 843, 846, 849, 850, 856, 857, 858, 861, 863, 865, 866, 867, 869, 871, 872, 873, 875, 877, 878, 880, 881, 890, 892, 908, 942, 943, 987, 1061]
[484, 519, 522, 525, 526, 528, 529, 530, 531, 535, 536, 537, 538, 539, 540, 543, 544, 545, 546, 547, 552, 553, 554, 555, 556, 557, 558, 562, 563, 564, 565, 566, 567, 568, 569, 571, 576, 578, 579, 586, 589, 591, 592, 593, 594, 595, 626, 640, 717, 737]
[30, 43, 69, 82, 85, 91, 99, 101, 304, 308, 314, 319, 325, 326, 327, 330, 333, 339, 343, 344, 350, 354, 357, 359, 361, 364, 369, 370, 374, 375, 379, 381, 390, 391, 392, 393, 394, 397, 402, 403, 407, 420, 423, 432, 435, 438, 439, 440, 518, 527, 590]
[10, 18, 28, 112, 114, 168, 277, 300, 315, 316, 320, 323, 324, 337, 340, 356, 368, 371, 376, 380, 382, 383, 385, 387, 388, 395, 399, 401, 404, 418, 421, 422, 424, 425, 428, 430, 434, 443, 450, 459, 463, 466, 467, 475, 476, 478, 481, 488, 493, 523, 542, 549, 

In [695]:
for df in [df20,df13,df8]:
    clus = df.columns.tolist()[:-1]
    f = open('output/'+str(len(clus))+'.txt','w')
    for clu_num in clus:
        for doc_id in df[df['vote']==clu_num].index.tolist():
            f.write(str(doc_id+1))
            f.write('\n')
        f.write('\n')
    f.close()

In [694]:
df8

,0,1,2,3,4,5,6,7,vote
0,0,0,0,0,10,3,5,3,4
1,0,0,0,0,10,3,5,3,4
2,0,0,0,0,10,3,5,3,4
3,0,0,0,0,10,3,5,3,4
4,0,0,0,0,10,3,5,3,4
5,0,0,0,0,10,3,5,3,4
6,0,0,0,0,10,3,5,3,4
7,0,0,0,0,10,3,5,3,4
8,0,0,0,0,10,3,5,3,4
9,0,0,0,0,10,3,5,3,4
